<a href="https://colab.research.google.com/github/ewilwertuoz/Work-with-Gensim-and-NLTK/blob/main/Bag_Of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF

In [ ]:
!pip install nmslib

In [ ]:
!pip install gensim

In [ ]:
import nltk
import gensim
import fitz
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
from gensim.similarities.nmslib import NmslibIndexer
from gensim.models import Word2Vec
from tempfile import mkstemp
from gensim import corpora
from nltk.stem import PorterStemmer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('stopwords')
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
stop_words = stopwords.words('english')
ps = PorterStemmer()
wnl = WordNetLemmatizer()

In [ ]:
path = "/content/drive/MyDrive/books/"

In [ ]:
with open(path + "all_file.txt", "r") as file:
    all_files = file.readlines()
file.close()

all_txt = []
for i in range(len(all_files)):
  all_files[i] = all_files[i].replace("\n", "")
  all_txt.append(all_files[i].replace(".pdf", ".txt"))

In [ ]:
def read_pdf(directory):
  doc = fitz.open(directory)
  txt = ""
  for current_page in range(len(doc)):
    page = doc.load_page(current_page)
    if current_page == 0:
      txt += page.get_text("text")
    else:
      txt += "\f" + page.get_text("text")
  txt = txt.replace("-\r", "").replace("-\n", "")
  return txt

In [ ]:
def add_lemma(sent):
  words = []
  words = [wnl.lemmatize(word) for word in simple_preprocess(sent, deacc=True)]
  return words

def add_stem(sent):
  words = []
  words = [ps.stem(word) for word in simple_preprocess(sent, deacc=True)]
  return words

In [ ]:
tokenized = []

rev_my_dictionary = {}

for file in all_files:
  text = read_pdf(path + file)
  for sentence in text.split('.'):
    tokenized.append(simple_preprocess(sentence, deacc = True))

  my_dictionary = corpora.Dictionary(tokenized)

  for key in my_dictionary.values():
    st_key = ps.stem(key)
    if st_key in rev_my_dictionary:
      if key not in rev_my_dictionary.get(st_key):
        rev_my_dictionary.get(st_key).append(key)
    else:
      rev_my_dictionary[ps.stem(key)] = [key]

tokenized.clear()

In [ ]:
tokenized = []

for file in all_files:
  text = read_pdf(path + file)
  for sentence in text.split('.'):
      tokenized.append(add_stem(sentence))

In [ ]:
model = Word2Vec(tokenized, min_count=1, epochs=100, seed=2)
indexer = NmslibIndexer(model)
_, temp_fn = mkstemp()
indexer.save(temp_fn)
new_indexer = NmslibIndexer.load(temp_fn)

In [ ]:
def search(item):
  elem = {}
  text = ""
  for txt in all_txt:
    nums = 0
    with open(path + "files/" + txt, "r") as fl:
      text = fl.read()
      fl.close()
    pages = text.split("\f")
    for page in pages:
      nums += 1
      if item in page:
        if txt.replace(".txt", "") in elem:
          elem.get(txt.replace(".txt", "")).append(nums)
        else:
          elem[txt.replace(".txt", "")] = [nums]
  return elem

In [ ]:
find = "intelligence"
stem_find = ps.stem(find)
all = model.wv.most_similar(stem_find, topn=10, indexer=new_indexer)
for i in range(len(all)):
  print(f"{all[i][0]} {all[i][1]}:")
  for item in rev_my_dictionary[all[i][0]]:
    dic = search(item)
    print(f"\t{item}:")
    for key in dic.keys():
      print(f"\t\t{key}: {dic.get(key)}")

In [ ]:
import os

file_names = os.listdir(path)

with open(path + "all_file.txt", "w") as file:
  for file_name in file_names:
      if file_name == ".ipynb_checkpoints" or file_name == "all_file.txt" or file_name == "files":
        continue
      file.write(file_name + "\n")
file.close()

In [ ]:
for file in all_files:
  text = read_pdf(path + file)
  with open(path + "files/" + file.replace(".pdf", ".txt"), "w") as fl:
    fl.write(text)